SDG Indicator comparison
===

In [1]:
#package
import os
import pathlib
import rasterio
import numpy as np
import geopandas as gp
from rasterstats import zonal_stats
from rasterio.plot import show

In [2]:
# Raster Reprojection
from Reprojection import reprojection

In [3]:
# Data Reprojection
srcpath = "data/sepal/Landsat/Jigawa/"
dstpath = "data/sepal/Landsat/Jigawa/reprojected/"
dstcrs = 'EPSG:26332'
# reprojection(srcpath, dstpath, dstcrs)

Boundary shapefile
===

In [4]:
boudary_shp_path = "data/boundary shapefile/Jigawa.shp" # from FAO GAU"
input_shapefile = gp.read_file(boudary_shp_path).to_crs(dstcrs)

Raster description
====


In [6]:
# Function for raster file descrition
def rasterdesc(path):
    """
    This function describe the raster depending on the raster path given:
    path: Raster path
    returns: raster description including the Nodata value and restoration label representative value
    """
    with rasterio.open(path) as src:
        array = src.read(1)
    num = np.unique(array)
    affine=src.transform
#     show(array)
    print('Image shape: ', src.shape)
    print('File path: ',src.name)
    print('Unique values in raster : ', num)
    print('Nodata:',num[0])
    print('Spatial Res:',affine[0])
    print('Degradation: %2d, Stable: %2d, Improvement: %2d' % (num[1],num[2],num[3]))

## Zonal Stats for sepal

In [7]:
def zonal_stat_sepal(raster, shapefile):
    """
    Input
    Raster: projected raster file
    shapefile: projected shapefile 
    
    Output: Categorical zonal stats
    """
    
    with rasterio.open(raster) as src:
        array= src.read(1)
        
        c_sepal = {1:'Degraded', 2:'Stable',3:'Improved'} #categorical rename
        affine = src.transform
        z_stat = zonal_stats(shapefile, array, affine=affine, categorical=True, category_map=c_sepal, nodata=0.0)
        import copy
        s_stat = z_stat[0]
        to_km = affine[0]**2/1e+6
        sepal_stat = s_stat.copy()
        sepal_stat.update((x, round(y*to_km,3)) for x,y in sepal_stat.items())
        return sepal_stat

In [8]:
#feed in the sepal data
# projected_ = dstpath

# rasterlst = [i for i in os.listdir(projected_) if i.endswith('tif')]
# se_result = []
# for raster in rasterlst:
#     path = projected_+raster
#     dicts =zonal_stat_sepal(path, input_shapefile)
#     se_result.append(dicts)

In [9]:
def sepal(srcpath,dstpath,dstcrs,input_shapefile):
    #reprojection
    reprojection(srcpath, dstpath, dstcrs)
    
    #for loop to pass each sub-indicator, compute and save the zonal stats in a list
    rasterlst = [i for i in os.listdir(dstpath) if i.endswith('tif')]
    se_result = []
    for raster in rasterlst:
        path = dstpath+raster
        dicts =zonal_stat_sepal(path, input_shapefile)
        se_result.append(dicts)
    return se_result

In [12]:
Jigawa_landsat =sepal(srcpath,dstpath,dstcrs,input_shapefile)

In [12]:
Jigawa_landsat = sepal(srcpath,dstpath,dstcrs,input_shapefile)
Jigawa_landsat

[{'Degraded': 1736.001, 'Stable': 21407.891, 'Improved': 681.382},
 {'Degraded': 21.504, 'Stable': 23766.044, 'Improved': 48.352},
 {'Degraded': 1695.612, 'Stable': 21436.658, 'Improved': 665.43},
 {'Degraded': 35.986, 'Stable': 23758.241, 'Improved': 5.036}]

In [13]:
srcpath = "data/sepal/Modis/Jigawa/"
dstpath = "data/sepal/Modis/Jigawa/reprojected/"
dstcrs = 'EPSG:26332'
boudary_shp_path = "data/boundary shapefile/Bauchi.shp" # from FAO GAUL"
input_shapefile = gp.read_file(boudary_shp_path).to_crs(dstcrs)
Jigawa_Modis =sepal(srcpath,dstpath,dstcrs,input_shapefile)
Jigawa_Modis

[{'Degraded': 646.407, 'Stable': 14458.896, 'Improved': 8890.627},
 {'Degraded': 21.397, 'Stable': 23937.708, 'Improved': 48.463},
 {'Degraded': 599.183, 'Stable': 14488.039, 'Improved': 8881.052},
 {'Degraded': 35.959, 'Stable': 23928.722, 'Improved': 5.097}]

In [20]:
srcpath = "data/sepal/Modis/Bauchi/"
dstpath = "data/sepal/Modis/Bauchi/reprojected/"
dstcrs = 'EPSG:26332'
boudary_shp_path = "data/boundary shapefile/Bauchi.shp" # from FAO GAUL"
input_shapefile = gp.read_file(boudary_shp_path).to_crs(dstcrs)

In [21]:
Bauchi_landsat =sepal(srcpath,dstpath,dstcrs,input_shapefile)
Bauchi_landsat

[{'Degraded': 9076.795, 'Stable': 39382.868, 'Improved': 367.741},
 {'Degraded': 63.195, 'Stable': 48649.027, 'Improved': 131.828},
 {'Degraded': 9023.652, 'Stable': 39494.6, 'Improved': 276.761},
 {'Degraded': 58.712, 'Stable': 48734.487, 'Improved': 0.73}]

## Zonal Stats for Trends Earth

**Reproject raster**  
Both sub-indicator and the SDG indicator raster

In [20]:
# Data Reprojection
srcpath = "data/Trend earth/Bauchi/"
dstpath = "data/Trend earth/Bauchi/reprojected/"
dstcrs = 'EPSG:26332'
reprojection(srcpath, dstpath, dstcrs)

Zonal stats for sub-indicator

In [34]:
TE_subindicator = "data/Trend earth/Bauchi/reprojected/Bauchi_prj.tif"

def te_subindicator_reclassifcation(raster):
    """
    The sub-indicator raster is a collection of multiple bands with different band index;
    Land cover change: band 12
    Productivity: band 7
    Soil Organic Carbon: band 21
    
    Each band has different classification scheme with different values(and different nodata value)
    
    Input Raster: path to the sub-indicator raster
    Output: A reclassified array collection of sub-indicators
    """
    sr = rasterio.open(raster)
    array = sr.read()

    band12 =sr.read(12)
    LUC = np.full(sr.shape, np.nan)
    LUC[band12 == -1] = -1
    LUC[band12 == 0] = 0
    LUC[band12 == 1] = 1

    band7 = sr.read(7)
    productivity = np.full(sr.shape, np.nan)
    productivity[(band7 > -32768) & (band7 <= -2)] = -2
    productivity[(band7 > -2) & (band7 <= 1)] = 1
    productivity[(band7 > 1) & (band7 <= 10)] = 10

    band21 = sr.read(21)
    SOC = np.full(sr.shape, np.nan)
    SOC[(band21 > -32768) & (band21 <= -10)] = -10
    SOC[(band21 > -10) & (band21 <= 9)] = 9
    SOC[(band21 > 9) & (band21 <= 100)] = 100

    TE_array = []
    TE_array.append(LUC)
    TE_array.append(productivity)
    TE_array.append(SOC)
    return TE_array

sub_indicators_array = te_subindicator_reclassifcation(TE_subindicator)

In [35]:
def te_subindicator_zonal_stats(raster,array, shapefile):
    """
    Compute zonal stats for TE sub-indicator
    raster: path to the sub-indicator
    array: TE_array pass in as a for loop
    shapefile: projected boundary shapefile
    
    Output: Categorical Zonal stats
    A for loop is needed to pass the array and result to be stored in a list
    """
    src = rasterio.open(raster)
    affine = src.transform
    c_trend = {np.unique(array)[0]:'Degraded', np.unique(array)[1]:'Stable',np.unique(array)[2]:'Improved'} #categorical rename
    z_stat = zonal_stats(shapefile, array, affine=affine, categorical=True, category_map=c_trend, nodata=-99999999)
    import copy
    t_stat = z_stat[0]
    to_km = affine[0]**2/1e+6
    te_stat = t_stat.copy()
    te_stat.update((x, round(y*to_km, 3)) for x,y in te_stat.items())
    return te_stat

te_result = []
for i in sub_indicators_array:
    result = te_subindicator_zonal_stats(TE_subindicator, i, input_shapefile)
    te_result.append(result)
    
te_result

[{'Degraded': 136.833, 'Stable': 48643.951, 'Improved': 234.37},
 {'Degraded': 5053.474, 'Stable': 23013.628, 'Improved': 20948.053},
 {'Degraded': 39.479, 'Stable': 48923.057}]

In [18]:
def TE_sdg_indicator(raster, shapefile):
    src = rasterio.open(raster)
    array= src.read(1)
    affine = src.transform
    c_trend = {np.unique(array)[1]:'Degraded', np.unique(array)[2]:'Stable',np.unique(array)[3]:'Improved'} #categorical rename
    z_stat = zonal_stats(shapefile, array, affine=affine, categorical=True, category_map=c_trend, nodata=-32768)
    import copy
    t_stat = z_stat[0]
    to_km = affine[0]**2/1e+6
    te_stat = t_stat.copy()
    te_stat.update((x, round(y*to_km, 3)) for x,y in te_stat.items())
    return te_stat

In [16]:
path = "data/Trend earth/Jigawa/reprojected/TE_Jigawa_prj.tif"
boudary_shp_path = "data/boundary shapefile/Jigawa.shp" # from FAO GAU"
input_shapefile = gp.read_file(boudary_shp_path).to_crs(dstcrs)
Jigawa_SDG_IndicatorTE = TE_sdg_indicator(path,input_shapefile)
Jigawa_SDG_IndicatorTE

{'Degraded': 6077.774, 'Stable': 12746.734, 'Improved': 5139.374}

In [21]:
path = "data/Trend earth/Bauchi/reprojected/Bauchi_sdg_prj.tif"
boudary_shp_path = "data/boundary shapefile/Bauchi.shp" # from FAO GAU"
input_shapefile = gp.read_file(boudary_shp_path).to_crs(dstcrs)
Bauchi_SDG_IndicatorTE = TE_sdg_indicator(path,input_shapefile)
Bauchi_SDG_IndicatorTE

{'Degraded': 24672.737, 'Stable': 16663.814, 'Improved': 7616.879}

Chart
========

In [16]:
import matplotlib.pyplot as plt